In [2]:
import digitalhub as dh
# conda install -c conda-forge gdal
PROJECT_NAME = "OPT_project"
proj = dh.get_or_create_project(PROJECT_NAME) # source="git://github.com/scc-digitalhu

In [3]:
%%writefile "opt_main_file.py"

from digitalhub_runtime_python import handler
import sys, os, time, json
import configparser
import argparse
from datetime import timedelta

# from multiprocessing import freeze_support, set_start_method #some stuff for multi-processing support
from joblib import Parallel, delayed, parallel_backend

from libs.RSdatamanager import filemanager as fm
from libs.RSdatamanager.Sentinel2.S2L2A import L2Atile, getTileList
from libs.RSdatamanager.Landsat.LandsatL2SP import L2SPtile, getL2SPTileList
from libs.ToolboxModules import featurext as m1


def tile_reading_1(tileDatapath, maindir, sensor):
    if sensor == 'S2':
        tile = L2Atile(maindir, tileDatapath)
    if sensor == 'Landsat':
        tile = L2SPtile(maindir, tileDatapath)
    return tile


def tile_reading_2(tile, outpath, tilename, year, **kwargs):
    #UPDATE OPTIONS
    name = str(tilename) + '_' + year
    update = {
        'year': year,
        'savepath': fm.check_folder(outpath, name)
    }
    #MODULE 1
    options = kwargs.get('module1',{})
    options.update( update )
    m1.manager(tile, **options)


def parallel_tile_reading(tiledict, maindir, sensor, tile_keys, outpath, tilename, years, **kwargs):
    tiles = Parallel(n_jobs=-1)(delayed(tile_reading_1)(tiledict[k], maindir, sensor) for k in tile_keys)
    Parallel(n_jobs=-1)(delayed(tile_reading_2)(tile, outpath, tilename, year, **kwargs) for tile in tiles for year in years)

@handler()
def main(project, datapath, options,module1):
    #PREPARE SOME TOOLBOX PARAMETERS
    sensor = options.get('sensor', None)
    tilename = options.get('tilename', None)
    years = options.get('years', None)
    maindir = options.get('maindir', None)
    outpath = options.get('outpath', None)
    deltemp = options.get('deltemp', True)

    with parallel_backend('loky'):
        if (module1):
            logging = {} 
            t_tot = time.time()  
            #READ DATASETS
            if sensor == 'S2':
                tiledict = getTileList(datapath)
            elif sensor == 'Landsat':
                tiledict = getL2SPTileList(datapath)
            else:
                raise IOError('Invalid sensor')
            keys = tiledict.keys()

            parallel_tile_reading(tiledict, maindir, sensor, keys, outpath, tilename, years, **kwargs)

            t_tot = timedelta(seconds=(time.time() - t_tot))     
            print("MOD1 TIME = ", t_tot,flush=True)
            logging['MODULE 1'] = {'TIME': str(t_tot) }
            with open(fm.joinpath(outpath,"logging_MODULE 1.txt"),'w') as json_file:
                json.dump(logging,json_file)

Writing opt_main_file.py


In [58]:
FUNCTION_NAME="execute-preprocessing"
func = proj.new_function(name=FUNCTION_NAME,
                         kind="python",
                         python_version="PYTHON3_9",
                         # ,"pydtw==2.0.2"
                         # requirements=["h5py","imageio","joblib","matplotlib","pandas","scikit-image","scikit-learn","six","scipy","numpy"],
                         handler="LattaruoloAndrea.optsar-fbk-test.main_dh_core:main",
                         code_src="git+https://github.com/LattaruoloAndrea/optsar-fbk-test")

In [ ]:
func.run(action="build", instructions=["echo '####################################################################'","conda install gdal","echo '####################################################################'"],)

In [59]:
options = {
    'sensor': "S2",
    'tilename': "42WXS",
    'years': ["2017","2018"],
    'maindir': "/",
    'outpath': "/outputs",
    'info': False,
    'deltemp': False
}

datapath = "/inputs"
m1options = True

input ={
    "datapath" : datapath,
    "options" :options,
	"module1" : m1options,
}

run_download = func.run(action="job",
                        #instructions=["echo '####################################################################'","conda install gdal","echo '####################################################################'"],
                        inputs={},
                         local_execution=False)

In [20]:
run_download

{'project': 'OPT_project', 'id': '052e3890-0dc5-492f-9765-0d1c03863ddc', 'kind': 'python+run', 'key': 'store://OPT_project/runs/python+run/052e3890-0dc5-492f-9765-0d1c03863ddc', 'metadata': {'project': 'OPT_project', 'name': '052e3890-0dc5-492f-9765-0d1c03863ddc', 'created': '2024-07-31T07:56:54.15Z', 'updated': '2024-07-31T07:56:54.163Z', 'created_by': 'tenant1userid', 'updated_by': 'tenant1userid'}, 'spec': {'task': 'python+job://OPT_project/execute-preprocessing:20b6ed6a-b732-4cc7-a8fa-63ef5bc96a7c', 'local_execution': False, 'source': {'source': 'git+https://github.com/LattaruoloAndrea/optsar-fbk-test', 'handler': 'LattaruoloAndrea.optsar-fbk-test.main_dh_core:main', 'lang': 'python'}, 'python_version': 'PYTHON3_9', 'function': 'python://OPT_project/execute-preprocessing:20b6ed6a-b732-4cc7-a8fa-63ef5bc96a7c', 'inputs': {}, 'outputs': {}, 'parameters': {}}, 'status': {'state': 'READY'}, 'user': 'tenant1userid'}